In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data from csv file
data = pd.read_csv("C:\\Users\\exx\\Downloads\\Rosemount_LeafOn_SfM_Canopies_PointVertexCoords.csv")

In [16]:
# list columns in data
print(data.columns)

Index(['treeID', 'area', 'diameter', 'volume', 'row', 'tree', 'xmin', 'ymin',
       'xmax', 'ymax', 'width', 'length', 'POINT_X', 'POINT_Y'],
      dtype='object')


In [17]:
# width = xmax - xmin (in UTM)
# length = ymax - ymin (in UTM)
new_data = data.copy()

# only keep the first row of each unique treeID
new_data = new_data.drop_duplicates(subset='treeID')

# drop POINT_X and POINT_Y columns
new_data = new_data.drop(columns=['POINT_X', 'POINT_Y'])

# print first 5 rows
print(new_data.head())


      treeID      area  diameter    volume  row  tree        xmin  \
0          1  0.702875  0.932928  0.917779    2     1  492091.600   
2375       2  0.580025  0.847863  0.286037    2     2  492094.270   
3466       3  2.625800  1.800605  3.096977    2     4  492098.905   
7393       4  0.496925  0.786473  0.268756    2     5  492102.285   
8510       5  0.783250  0.991370  0.481919    2     6  492105.190   

             ymin        xmax         ymax  width  length  
0     4952560.805  492092.745  4952562.130  1.145   1.325  
2375  4952561.230  492095.345  4952562.315  1.075   1.085  
3466  4952560.360  492100.990  4952562.745  2.085   2.385  
7393  4952561.120  492103.435  4952562.060  1.150   0.940  
8510  4952560.830  492106.180  4952562.295  0.990   1.465  


In [18]:
# calculate diagonal width, length, and diameter for each tree
new_data.loc[:, 'diagonal_width'] = np.nan
new_data.loc[:, 'diagonal_length'] = np.nan
new_data.loc[:, 'diameter'] = np.nan

for treeID in new_data['treeID']:
    tree = data[data['treeID'] == treeID].copy()
    tree.loc[:, 'x_ymin'] = min(tree.loc[tree['POINT_Y'] == tree['POINT_Y'].min(), 'POINT_X'])
    tree.loc[:, 'x_ymax'] = max(tree.loc[tree['POINT_Y'] == tree['POINT_Y'].max(), 'POINT_X'])
    tree.loc[:, 'y_xmin'] = min(tree.loc[tree['POINT_X'] == tree['POINT_X'].min(), 'POINT_Y'])
    tree.loc[:, 'y_xmax'] = max(tree.loc[tree['POINT_X'] == tree['POINT_X'].max(), 'POINT_Y'])
    tree.loc[:, 'diagonal_width'] = np.sqrt(((tree['xmax'] - tree['xmin'])**2 + (tree['y_xmax'] - tree['y_xmin'])**2))
    tree.loc[:, 'diagonal_length'] = np.sqrt(((tree['ymax'] - tree['ymin'])**2 + (tree['x_ymax'] - tree['x_ymin'])**2))

    # find diameter (this takes a while to execute)
    max_distance = 0
    for i in range(len(tree)):
        for j in range(i+1, len(tree)):
            # diameter is longest euclidean distance between any two points in polygon
            distance = np.sqrt((tree.iloc[i]['POINT_X'] - tree.iloc[j]['POINT_X'])**2 + (tree.iloc[i]['POINT_Y'] - tree.iloc[j]['POINT_Y'])**2)
            if distance > max_distance:
                max_distance = distance

    new_data.loc[new_data['treeID'] == treeID, 'diameter'] = max_distance
    new_data.loc[new_data['treeID'] == treeID, 'diagonal_width'] = tree['diagonal_width'].values[0]
    new_data.loc[new_data['treeID'] == treeID, 'diagonal_length'] = tree['diagonal_length'].values[0]
    new_data.loc[new_data['treeID'] == treeID, 'x_ymin'] = float(tree['x_ymin'].values[0])
    new_data.loc[new_data['treeID'] == treeID, 'x_ymax'] = float(tree['x_ymax'].values[0])
    new_data.loc[new_data['treeID'] == treeID, 'y_xmin'] = float(tree['y_xmin'].values[0])
    new_data.loc[new_data['treeID'] == treeID, 'y_xmax'] = float(tree['y_xmax'].values[0])

In [19]:
print(new_data.head())

      treeID      area  diameter    volume  row  tree        xmin  \
0          1  0.702875  1.559046  0.917779    2     1  492091.600   
2375       2  0.580025  1.285972  0.286037    2     2  492094.270   
3466       3  2.625800  2.514806  3.096977    2     4  492098.905   
7393       4  0.496925  1.240363  0.268756    2     5  492102.285   
8510       5  0.783250  1.478792  0.481919    2     6  492105.190   

             ymin        xmax         ymax  width  length  diagonal_width  \
0     4952560.805  492092.745  4952562.130  1.145   1.325        1.321599   
2375  4952561.230  492095.345  4952562.315  1.075   1.085        1.095320   
3466  4952560.360  492100.990  4952562.745  2.085   2.385        2.345815   
7393  4952561.120  492103.435  4952562.060  1.150   0.940        1.208056   
8510  4952560.830  492106.180  4952562.295  0.990   1.465        1.015246   

      diagonal_length      x_ymin      x_ymax       y_xmin       y_xmax  
0            1.388470  492092.120  492092.535  4

In [20]:
# save df to csv
new_data.to_csv("C:\\Users\\exx\\Downloads\\Rosemount_SfM_Canopies_Euclidean_HW.csv", index=False)